In [1]:
#imports
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text


2023-07-20 13:51:12.164476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = '/Users/patrick/Desktop/Lighthouse_labs/Data Projects/Ted-Talks-Recommender-System/Data_output/ted_talk_clean_merged_bert.csv'
df = pd.read_csv(data, index_col=0)
df.head()

,author,title,description,likes,views,transcript,date,tags
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio...",Jun 2023,"art, creativity, design, fashion, performance"
1,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...,Jun 2023,"diversity, identity, inclusion, indigenous_peo..."
2,Rebecca Darwent,how to fund real change in your community,is there a way to give back that benefits ever...,1000,341218,I spent my whole career in the nonprofit secto...,Jun 2023,"business, community, equality, humanity, money..."
3,Susanne Buckley-Zistel,what caused the rwandan genocide,for one hundred days in 1994 the african count...,3700,126376,"For 100 days in 1994, the African country of R...",Jun 2023,"africa, animation, education, history, identit..."
4,Conor Russomanno,a powerful new neurotech tool for augmenting y...,in an astonishing talk and tech demo neurotech...,1100,374259,I became obsessed with the relationship betwee...,Jun 2023,"biotech, brain, disability, health, invention,..."


In [3]:
import regex as re

In [4]:
df['transcript'] = df['transcript'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [5]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)

In [6]:
import tensorflow as tf

In [7]:
def get_bert_embeddings(text, preprocessor, encoder):

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    encoder_inputs = preprocessor(text_input)
    outputs = encoder(encoder_inputs)
    embedding_model = tf.keras.Model(text_input, outputs['pooled_output'])
    sentences = tf.constant([text])
    return embedding_model(sentences)

In [8]:
df['encodings'] = df['transcript'].apply(lambda x: get_bert_embeddings(x, preprocessor, encoder))

In [24]:
def preprocess_text(text):
  # text = input()
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text

In [15]:
# get transcript from df where author is the input
def get_transcript(df, author):
    transcript = df.loc[df['author'] == author, 'transcript'].values[0]
    return transcript

def get_transcript_list(df, author):
    transcript_list = df.loc[df['author'] == author, 'transcript'].values
    return transcript_list

In [29]:
text = get_transcript(df, 'Barry Schwartz')
text

'Hello everybody I m honored to be here to talk to you and what I m going to talk about today is luck and justice and the relation between them Some years ago a former student of mine called me to talk about his daughter It turns out his daughter was a high school senior was seriously interested in applying to Swarthmore where I taught and he wanted to get my sense of whether she would get in Swarthmore is an extremely hard school to get into So I said Well tell me about her And he told me about her what her grades were like her board scores her extracurricular activities And she just sounded like a superstar wonderful wonderful kid So I said She sounds fabulous She sounds like just the kind of student that Swarthmore would love to have And so he said Well does that mean that she ll get in And I said No There just aren t enough spots in the Swarthmore class for everybody who s good There aren t enough spots at Harvard or Yale or Princeton or Stanford There aren t enough spots at Google

In [30]:
query_text = preprocess_text(text)
query_encoding = get_bert_embeddings(query_text, preprocessor, encoder)

In [11]:
# import cosine similarity from sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
df['similarity_score'] = df['encodings'].apply(lambda x: cosine_similarity(x, query_encoding)[0][0])
df_results = df.sort_values(by=['similarity_score'], ascending=False)

In [32]:
df_results.head(5)

,author,title,description,likes,views,transcript,date,tags,encodings,similarity_score
1081,Barry Schwartz,what role does luck play in your life,chance plays a far bigger role in life than we...,1800,618885,Hello everybody I m honored to be here to talk...,Sep 2020,"equality, personal_growth, psychology, social_...","((tf.Tensor(-0.96350807, shape=(), dtype=float...",1.000000
2038,Isadora Kosofsky,intimate photos of a senior love triangle,photographer and ted fellow isadora kosofsky i...,1600,558698,Jeanie Will and Adina are three senior citizen...,Sep 2018,"aging, art, culture, humanity, identity, love,...","((tf.Tensor(-0.9765004, shape=(), dtype=float3...",0.980486
4611,Chip Conley,measuring what makes life worthwhile,when the dotcom bubble burst hotelier chip con...,10300,3454658,I m going to talk about the simple truth in le...,Jun 2010,"buddhism, business, economics, happiness, mone...","((tf.Tensor(-0.9888299, shape=(), dtype=float3...",0.980300
5102,Jill Sobule + Julia Sweeney,the jill and julia show,two ted favorites jill sobule and julia sweene...,1600,556579,Jill Sobule At a conference in Monterey by th...,Feb 2008,"collaboration, comedy, entertainment, humor, m...","((tf.Tensor(-0.98089206, shape=(), dtype=float...",0.980127
5145,Richard Branson,life at 30000 feet,richard branson talks to teds chris anderson a...,5900,1986145,Chris Anderson Welcome to TED Richard Branson ...,Oct 2007,"business, entertainment, entrepreneur, flight,...","((tf.Tensor(-0.98741657, shape=(), dtype=float...",0.979948
